# Recommender System Mini-Project

The main purpose of this notebook is to show the results of two recommender systems that were developed to help Deezer, a streaming music service, improve its song recommendations for its users.

This notebook has the following structure
* Introduction: Business Objective and Data Set
* Approach: Sampling and Preprocessing Data
* Validation and Testing Set-up
* Neighborhood Collaborative Filtering
* Latent Factors Model
* Conclusions: Results and Open Questions


## Imports

In [1]:
# Managing relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models.model_based.matrix_creation import *
from models.cross_validation.ModelTester import ModelTester
from models.cross_validation.parameter_test import parameter_test, ready_to_plot
from models.model_based import matrix_factorization_model
from models.neighborhood_based.ItemBasedCF import ItemBasedCF
from utils.similarity import ochiai, cosine
import utils.loss_functions as lf

In [3]:
%matplotlib inline

-----

## Introduction: Business Objective and Data Set

<img src="./images/logo.png">

**Deezer** is a French music streaming service. It proposes more than 43 million tracks and is available in more than 180 countries through both a free limited service and a premium offer. **It differentiates from the competition by placing a greater emphasis on personalizing song recommendations to users**. This is mainly done by two complementing strategies. On one hand, the Editors *hand-pick* the content that they feel would be more relevant for the users and, on the other hand, Deezer's *analytics* determine which users would enjoy this curated content the most.

The core features that Deezer employs to deliver its recommendations are **Flow** and **Discovery Radio**. We only focus on the former since the data provided only deals with that feacture. **The base concept of Flow is quite simple: provide an *endless* stream of recommendations which adaptively responds (and learns) from user explicit and implicit feedback**. For example, if a song was explicitly disliked, then the algorithm knows that it should never be streamed again. Or, on a more positive note, if a song was listened for over 30 seconds then the algorithm will embed this implicit information into a user profile. This profile would then provide a similarity notion within the users and therefore bridge recommendations between them. 

As has been discussed so far, **it intuitively seems appealing that any Recommender System that improves the accuracy of Flow's suggestions would improve Deezer's user experience**. Although to a certain degree this is true, **there is also a great risk of polarizing the recommendations** to the safe *favorite songs* of the users **and hence endanger novelty and serendipity** which, at the end, are the central value propositions of Deezer. Moreover, Flow's accuracy can only be improved to up to a certain level.
Since the recommendations from an *endless* stream of suggestions, contrary to a fix list of songs as in Spotify, then the accuracy is condemn to account for the natural *trial and error*s. In other words, since Deezer needs to continuously provide recommendation on Flow, it is bounded to make more mistakes as the time passes.

To construct on the previous discussion, let's define Flow's hit rate as the number of songs that were listened by a user divided over the total number of songs that were exposed during the session. **Due to the emphasis on novelty, serendipity and also to the continuous steam of suggestions, we would not expect a high accuracy, but rather something closer to 70% or even 60%**. This is a great example of how having a lower accuracy is something desirable; it suggest that Deezer is testing new recommendations rather than playing only safe bets. Nonetheless, Deezer's Recommendation System does have an opportunity of improvement. **We have observed that there is significant variance on the accuracy between user groups and also that the accuracy plummets for less active users** (as seen in the next plot).

Before jumping into that discussion, here is a technical note on how the hit rate is approximated in the data. **Deezer defines that a song was listened on Flow as long as the song was played for over 30 seconds** and not listened otherwise. The hit rate would then be computed as the number of songs that were listened for over 30 seconds divided by the total number of songs that were exposed throughout the data's time frame. **We do agree that if a user evidently hates a song he is bound to skip it** within the first 30 seconds. Therefore, Deezer's definition does unveil the strong dislikes of its user base. **However, we are cautious on inferring the opposite**. That is, that the user liked a recommendation because it was played for over 30 seconds. To make this inference we would narrow down our scope to the *explicitly* liked songs. That is, the songs that received a *heart* when they were played on Flow. However, this is not available in our data set. **Thus, we are going to infer that a user liked the recommendation as long as it was listened for over 30 seconds**. *Note: an in depth discussion of the data set is presented in the next section.*

<img src="images/hr_evolution.png">

**The plot is read as follows**. On the y-axis is Flow's average hit rate per user. That is, the hit rate averaged over all the songs that were exposed to the user on Flow during the data's time frame. On the x-axis is the total number of users taken into consideration when computing the general hit rate. To move from left to right, we add the most relevant users that are outside of our current sample. In other words, when users = 1000 then the sample contains the 1k most active users, when users = 2000 then it contains the 2k most active users (including the previous user subset) and so on.

As it can be seen, **Flow does a better job at recommending songs to the most relevant users but its quality of recommendation worsens for the less active users**. This seems reasonable since Flow has more interactions to better learn from the most active users.
However, where we find an opportunity of improvement is on smoothing the rate of decay of the hit rate to a more steady pattern. **Our main hypothesis for this decaying curve is that Flow's Recommender System (intentionally or unintentionally) tends to experiment more with users that have less activity history**. 

**Our approach**, in principle, **would then focus on playing safer bets instead of gambling with the new users**. Where playing safer bets means recommending songs that have proven successful for a wide base of users. Contrary to recommending songs that are only listened by a small subset of users. This with the final purpose of flattening out the hit rate curve displayed above. **Our main hypothesis and approach find justification from the following plot**.

<img src="images/exposure_evolution.png">

Again, the x-axis represents the number of most active users that are added into the sample. Now the y-axis represents the average number of times that a song is exposed on Flow. **The fast drop of the average is congruent with our hypothesis**. It proves that Flow displays a larger set of songs to those users than to more active users. 

<img src="images/add.png">

**The above plot also strengthens our hypothesis**. It shows that the most relevant songs in the data set are already uncovered by the 1k most active users. Therefore, there is no clear justification of why Flow is exposing a broader set of songs to the less active users. **If, given a subset of songs, Flow is able to provide novel or serendipitous recommendations to the most active users, then that selection of songs should perform equally well for the less active users**. This holds because as seen in the graph, the less active users mostly listen to the same songs as the active users.

**The plot was computed as follows**. First, for each user group, we determined the relevant set of songs that account for 80% of the total (user, song) interactions in that sample. Then, given the list per each user group, we calculated the number of intersections between the two adjacent user groups divided by the total of the larger group. Using some notation, if $I_{i}$ denotes the set of most relevant songs on user group i and $I_{i+1}$ the set of most relevant songs for group i+1 then the y-axis plots the evolution of $$\frac{|I_{i} \cap I_{i+1}|}{|I_{i+1}|}$$ as more users are added to the sample.

**To close this section we will describe the data set that was proportioned by Deezer** and then move into the following sections to discuss our approach to improve Flow's recommendations based on the insights discussed so far.

In [4]:
# Make sure to specify the right path to load the data
path = '/home/ap/personalization_project/models/model_based/train.csv'
data = pd.read_csv(path)

The variables available in the dataset are the following:

In [5]:
data.columns

Index(['genre_id', 'ts_listen', 'media_id', 'album_id', 'context_type',
       'release_date', 'platform_name', 'platform_family', 'media_duration',
       'listen_type', 'user_gender', 'user_id', 'artist_id', 'user_age',
       'is_listened'],
      dtype='object')

**The data set tracks the (user, song) combinations that occured in a given month**. The identifiers of users and songs are on the **"user_id"** and **"media_id"** columns respectively. Moreover, there are two columns of interest that we use to compute Flow's hit rate: **"is_listened"** and **"listen_type"**. 

The variable **"listen_type"** takes the value 1 if the song was played on Flow and 0 otherwise. On the other hand, the variable **"is_listened"** takes the value 1 if the song was listened for more than 30 seconds and 0 otherwise. Note that all combinations of (**"listen_type"**, **"is_listened"**) are possible. For example, (0, 0) is interpreted as a song that was listed to by less than 30 seconds outside of Flow (maybe a user flipping through a playlist to find a certain song). And (1, 1) would be interpreted as a success on Flow. In this case a song was displayed on Flow and the user listened to it for more than 30 seconds. Therefore, the hit rate is calculated as the amount of times that **"is_listened"** = 1 (sum of **"is_listened** = 1 when **listen_type"** = 1) divided by the amount of times that a song was exposed on Flow (that is the count of **"listen_type"** == 1) 

Therefore, **we are trying to predict the probability that a song will be listened on Flow given the observations that we have on the hit rates**

In [6]:
data.shape

(7558834, 15)

In [7]:
data.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,0,9241,55164,29,0
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,0,16547,55830,30,1
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,1,7665,2704,29,1
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,1,1580,938,30,0
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,1,1812,2939,24,1


Currently, the rest of the variables are not used in our Collaborative Filtering approach. But in general they provide additional characterization on users, songs and characteristics of the moment when the song was listened to. To give some examples, for users we have age and gender, for songs we have the artist, release date and genre and, finally, for the moment characterization, we have hour of the day, the device that was used (computer, cellphone) and the context (playlist, album or Flow).

---

## Approach: Sampling Data and Preprocessing

As discussed in the past section, our approach aims to flatten the hit rate curve that was observed in Deezer's data. In order to do so, **we first strategically sample the data and then select the Collaborative Filtering model that fits best**. The criteria for selecting the data will prioritize certain songs that are relevant but preserve novelty and serendipity. On the other hand, we build two intrinsically different CF models: one neighborhood-based and one model-based. The first model would in principle have the highest quality of prediction at the expense of a higher running time and possibly not generalizing properly due to overfitting. The second model would yield more general predictions but at a lower running time. **We build two models with the purpose of selecting the one that most accurately fits the specific sample but also to understand the different insights that each of them brings about the data**.

Apart from doing the standard Machine Learning train/test cross-validation schemes, **we also compare our results against three different baseline models and two industry standard built-in models (SVD and SVD++)** from the `scikit-surprise` package. In the next section we will show how tuning our parameters specifically to the problem at hand provides a higher accuracy than simply running the aforementioned models on the data.

The raw data has close to **~20k users** and **~450k songs**. 

In [8]:
# The number of unique users and items in the raw data
data['user_id'].nunique(), data['media_id'].nunique()

(19918, 452975)

Apart from lacking the computer power to run the models on that size of data. We opted to trim the number of users by half and work with **10k users** since we observed a long tail in the data. As seen below, close to **35%** of the users (**~7k**) amount to **80%** of the total (user, song) interactions. Therefore we do not lack representativeness by focusing on this subset of users. 

<img src="images/longtail.png">

### Sampling Strategy

As discussed in the first section, our main hypothesis is based on Flow's gambling behavior with the less active users. In order to mitigate this, right from the start, we are going to **fix the number of songs** fed into our Recommender Systems in order to avoid forcing them to predict probabilities for all (user, song) combinations. This leads to a lower change of exposing users to songs that might not be attractive. This approach might seem to put novelty and serendipity at risk. However, as seen in the following graph, it can be argued otherwise.

<img src="images/song_evolution.png">

**The previous plot shows that restricting our analysis to a fixed amount of relevant songs does not inhibit the possibility of providing some novel or serendipitous recommendations**. This is the case because the most popular songs tend to polarize the majority of (user, song) interactions in the data. Therefore **to provide a new recommendation to a user we do not have to necessarily look into the space of all the songs but we can rather come up with an appealing recommendation from the subset of the most relevant songs**. Of course this strategy affects the users whom mostly listen to songs that are not in the list of most popular songs. In our approach, those users will receive *mainstream* recommendations on Flow. However this a different problem on its own. And our approach focuses on creating a less variant hit rate for the less active users.

The most relevant songs were computed as the amount of songs that accounted for more than a specified % of interactions in the data. Due to computational limitations we chose a 10% threshold which yields the results of the above graph. We are sure that a higher threshold would provide a wider variety of results, however the 10% threshold allowed us to experiment with different parameter values in our models (in a reasonable amount of time) and still yield a high quality of prediction.

### Sampling Technical Details

To close this section we provide the technical details on how our sampling was constructed. **In general our sampling uses the hit rate as an implicit proxy of the likelihood that a user might listen to a song on Flow**. Since some (user, song) combinations occur more than one time in the data, we use the median to aggregate those repetitions. We chose the median since it will factor out sporadic 0 hit rates that might occur when the user skips a song that he likes because he is not in the mood for it. However, if those 0 hit rates become more prevalent then they will directly affect the aggregation.

**We followed two sampling approaches: the *only-popular-items* approach and the *50/50 approach ***. In both approaches we first filter out the users that do not have sufficient song ratings (over 10). This to avoid making inferences with users that are fairly new to the platform. In the first approach we narrowed our analysis to the 100 most relevant songs in the data. This gave us a fill-rate on the (user, song) matrix of approximately 30%. On the second approach again we narrowed the analysis to a 100 songs but this time half of the sample was taken from the 50 most relevant songs and the rest at random from the remaining songs. This approached yielded a lower fill-rate of 20% but still sufficient to run our analysis. All the functions used for the sampling as well as the computations discussed are presented below.

In [9]:
# Most popular items
ratings = hit_rate_matrix_popular_items(data)
# 50% popular, 50% less popular
ratings2 = hit_rate_matrix_50_50(data)

Below are also the user-item matrix fill-rates that result from our sampling schemas.

In [10]:
# Shape of the rating matrix with 100 most encountered items
ratings.shape

(8644, 100)

In [11]:
# Fill rate of the matrix with the 100 most encountered items
ratings.count().sum() / float((ratings.shape[0] * ratings.shape[1]))

0.27109208699676074

In [12]:
# Shape of the rating matrix with 50 most encountered items and 50 randomly sampled
ratings2.shape

(6224, 100)

In [13]:
# Fill rate of the 50/50 matrix
ratings2.count().sum() / float((ratings2.shape[0] * ratings2.shape[1]))

0.20729113110539846

---

## Validation and Testing Set-up

The framework for testing our results follows the classic Machine Learning approach of splitting the data into train, validation and test samples given a set of proportions (our standard is 70% train, 20% validation and 10% test).

**To implement this framework, we developed a class: `ModelTester`**. **This class first creates the three data splits and then evaluates the user specified model for any given loss function**. In order to make the predictions independent on the train/validation splits, the class has a built-in cross-validation methodology.
It works as follows. For a fixed test set, the methodology interchanges at random the elements from the train and validation set and then it re-evaluates the model storing the new results. It eliminates the dependency by taking the mean over all the results and also tracks of the variability by computing the standard deviation.

Below is an example of how ModelTester is employed

In [14]:
cross_validation = ModelTester()

The attributes of the `ModelTester` instances are the following:

In [15]:
cross_validation.__dict__

{'MODEL_BASED': True,
 'data': None,
 'non_null_indices': [],
 'ratios': (0.7, 0.2, 0.1),
 'seed': 42,
 'test_set': {},
 'train_set': {},
 'valid_set': {}}

Below is an example of how the method `fit_transform()` creates the different splits of the data

In [16]:
# Fit the instance to our data, it will create the different sets
cross_validation.fit_transform(ratings2)

>>> The cross-validation framework is being built ...
    Please wait, you'll be able to use all of its features soon!
>>> DONE


In [17]:
len(cross_validation.train_set), len(cross_validation.valid_set), len(cross_validation.test_set)

(90312, 25804, 12901)

Apart from testing our models against the data, we compare them to baseline models. These models are constructed in three different ways:
- Assigning a **constant** hit rate of 1 to every user-item pair
- Assigning a **random** hit rate to every user-item pair
- Assigning the **mean** hit rate to every user-item pair

Following are the results in terms of MSE and AME

**Constant Hit Rate**

In [18]:
# Assining a hit rate of 1 to every user-item pair
baseline = pd.DataFrame(np.ones(ratings2.shape))
baseline.columns = ratings2.columns
baseline.index = ratings2.index

# Get metrics: MSE and AME
cross_validation.evaluate_valid(baseline, loss_func=lf.mean_squared_error)
cross_validation.evaluate_valid(baseline, loss_func=lf.absolute_mean_error)

MSE: 0.371986528217
AME: 0.423658216803


0.42365821680337878

**Random Hit Rate**

In [19]:
# Assining a random hit-rate to every user-item pair
baseline = pd.DataFrame(np.random.rand(*ratings2.shape))
baseline.columns = ratings2.columns
baseline.index = ratings2.index

# Get metrics: MSE and AME
cross_validation.evaluate_valid(baseline,loss_func=lf.mean_squared_error)
cross_validation.evaluate_valid(baseline,loss_func=lf.absolute_mean_error)

MSE: 0.282780989073
AME: 0.449345378298


0.44934537829756288

**Mean Hit Rate**

In [20]:
# Assinging the mean hit-rate to every user-item pair
baseline = pd.DataFrame(np.ones(ratings2.shape)) * ratings2.mean().mean()
baseline.columns = ratings2.columns
baseline.index = ratings2.index

# Get metrics: MSE and AME
cross_validation.evaluate_valid(baseline,loss_func=lf.mean_squared_error)
cross_validation.evaluate_valid(baseline,loss_func=lf.absolute_mean_error)

MSE: 0.192710839643
AME: 0.405227227455


0.40522722745462147

It is worth pointing out **why the Mean Hit Rate algorithm performs rather well on the data**. For a song to be part of the most relevant elements it is expected that it has some common appeal between the users. Therefore, its hit rate most likely resembles a normal distribution where the mass concentrates around the common perception of the song by the users. The good prediction of this algorithm is then the result of the mean being an unbiased estimator when sampling from a normal distribution. This in contrast to the Random Hit Rate algorithm where the assumption in this case is that there is no common agreement between the users and hence the hit rate most likely follows a uniform distribution.

**Next are the results that we obtained from running `scikit-surprise` SVD and SVD++ models**. In the first test we ran the simplest SVD model: no biases or regularization. For our second test we ran the full SVD++. Following are the results from these experiments. 

<img src="images/svd.png">

As it can be seen from the previous graph, the best predicion error for the SVD model does not significantly vary when k increases. However, MSE is bounded below by an MSE of 0.19.

<img src="images/svdpp.png">

The results for the SVD++ are almost equivalent to the SVD. However, the MSE is a bit better but still bounded below by 0.19.

**Both package models performed barely above the baseline models and**, as it will be seen in the coming sections, **are outperformed by our models**.

---

## Neighborhood Collaborative Filtering

This section is organized as follows. First we provide a brief description of how to use the class that was developed to implement this model. Then, in terms of hyperparameter tuning we discuss the effect of varying the value of k (the number of neighbors) in the predictions of the model for the two sampling schemes discussed in the past sections. Finally, we investigate how scaling impacts the running time of this model.

In terms of the framework for the model, we opted for an **item-based** CF model since the number of ratings per song in the data is greater then the number of ratings per user. Hence, adopting this approach would allow us to have more accurate similarity measures when comparing two songs.

We designed a **class** called `ItemBasedCF` that predicts the probability that any (user, song) pair would be listened on Flow. This class was coded from scratch which, as it would be seen in the last section, allowed us to have achieve a fast running time. This is the case because the code was specifically developed to perform only the methods in the class and also to use the memory efficiently while running the algorithm.

The class works as follows: first we create an instance of the class. We need to specify two elements:
- the number `k` of neighbors to use when calculating the estimated rating
- the measure of similarity `sim_measure` to use to compute the similarity between items.

In [21]:
# We create an instance with k=15 and the Ochiai similarity measure (cosine for binary inputs)
ibcf = ItemBasedCF(k=15, sim_measure=ochiai)

In [22]:
# Here are its attributes
ibcf.__dict__

{'data': None,
 'items': [],
 'k': 15,
 'memory': {},
 'sim_measure': <function utils.similarity.ochiai>,
 'users': []}

We then `fit` the data to our rating matrix: it will set the data and the list of users and the list of items available as attributes of the instance.

In [23]:
ibcf.fit(ratings2)

Now, one can predict any combination (user, song) pair by calling the `predict` function.

In [24]:
ibcf.predict(user=1, item=3135183)

0.65430456185270292

Here it is the **predicted hit-rate** for the pair `(user, item) = (1, 3135183)`. If we want to binarize the ouput we should simply choose a threshold and then apply it to the prediction (a natural threshold would be `0.5`).

### Tuning parameter k

To evaluate neighborhood-based models we need to **hide** some ratings that are present in the original matrix in order to compare them with the predictions that the model will do for those missing entries. 
All this work of hiding and comparing is done in the `ModelTester` class that we created. In particular, the class method `ModelTester().shuffle_cv()` will take care of creating the different train sets.

We need to use the function `parameter_test` to perform the cross validation. For this we specify the values of the parameter `k` we want to test and the number `cv_times` of folds for the cross validation.

In [25]:
k_vals = [10, 20, 30]
cv_times = 2

Since the number of k values to test and the number of folds require the model to be run many times, we have constructed a smaller rating matrix for the reader to be able to try the function and see the results much faster

In [26]:
# We reduce the rating matrix to 1000 user only
smaller_ratings = ratings[:1000]

In [27]:
smaller_ratings.head()

media_id,983248,1575269,3135183,5639256,8086130,12565420,14954267,61142290,61752797,62724019,...,132625720,132929042,133102516,133165774,133637516,133661814,133940962,134711946,134748108,135736388
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,1.0,NaN,NaN,1.000000,1.000000,1.0,1.000000,1.0,1.000000,...,1.0,NaN,0.937500,1.00,NaN,1.000000,1.0,1.0,NaN,NaN
1,1.0,1.0,NaN,NaN,1.000000,0.500000,NaN,0.666667,1.0,1.000000,...,1.0,NaN,0.800000,NaN,NaN,0.833333,1.0,1.0,NaN,NaN
2,1.0,1.0,NaN,1.0,1.000000,0.928571,1.0,1.000000,1.0,0.923077,...,NaN,NaN,0.909091,0.00,NaN,0.000000,0.0,0.0,NaN,NaN
4,NaN,0.0,NaN,NaN,0.333333,NaN,0.0,NaN,0.0,NaN,...,0.0,0.0,0.000000,0.75,0.0,0.800000,NaN,NaN,0.0,0.0
5,1.0,1.0,1.0,NaN,1.000000,1.000000,NaN,1.000000,1.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,1.0,NaN,NaN


In [28]:
result_test, result_train = parameter_test(k_vals,  
                                           ItemBasedCF, 
                                           ModelTester(), 
                                           lf.mean_squared_error, 
                                           smaller_ratings,
                                           cv_times,
                                           verbose=True)

>>> The cross-validation framework is being built ...
    Please wait, you'll be able to use all of its features soon!
>>> DONE
----------- k = 10 -----------

Test set
MSE: 0.127354803973
>>> Fold 0:

Train set
MSE: 0.121100883422
>>> Fold 1:

Train set
MSE: 0.123389679531
>>> The cross-validation framework is being built ...
    Please wait, you'll be able to use all of its features soon!
>>> DONE
----------- k = 20 -----------

Test set
MSE: 0.116917851198
>>> Fold 0:

Train set
MSE: 0.112161348274
>>> Fold 1:

Train set
MSE: 0.113890235414
>>> The cross-validation framework is being built ...
    Please wait, you'll be able to use all of its features soon!
>>> DONE
----------- k = 30 -----------

Test set
MSE: 0.109323620232
>>> Fold 0:

Train set
MSE: 0.110465680658
>>> Fold 1:

Train set
MSE: 0.111645107821


We observe in the above results that the greater the value of `k`, the smaller the loss. This past statement applies only for a subset of the data that was used, though. Below are the results obtained fot the **whole rating matrix**. Since it takes over 30 min to run, we have computed and stored the outcomes separately and only present the most interesting results in this notebook

<img src="images/CF_HR_N.png">

The previous plot shows the performance of our Item-based model on both the train and test sets for the first sampling schema. **The main result of this graph is that to learn accurate predictions for the 100 most relevant songs, we need to *at least* consider the song's 40 closest neighbors**. Increasing the number of neighbors does not provide any additional information but decreasing its number does worsens the quality of the recommendation. We understand that it is too restrictive to focus on 40 neighbors when the total number of songs is 100. However, note that even for k = 10 we achieve better predictions than any of the baseline and industry models.

In terms of flattening out the hit rate curve for Deezer, **the key take away is the size of the threshold that should be considering before exposing a new song to the user**. If the song has over 40 neighbors in the set of songs that have been listened by the user, then it is almost guaranteed to be a correct prediction. On the contrary, if a song does not have many neighbors present then it becomes a riskier bet to expose it on Flow.

<img src="images/CF_5050_2.png">

Again this plot shows the performance of the Item-based model but now on the second sampling schema. **We observe a similar pattern** as in the previous graph, where considering more than 40 neighbors does not provide any additional information. Since now 50 songs are taken at random, this plot seems to conjecture that this pattern might be independent of the relevancy of the songs but rather on the number of total items consider. To enrich the analysis, a next step would be increment the number of items and see whether the optimal number of neighbors increases or stays the same.

**The past two plots suggest an approach of how Deezer can deal with the cold-start problem (for new songs)**. That is, to find whom are the best users to present the new material from artists. Since a consensus between 40 songs is a safe requirement for high prediction then Deezer could leverage all of the songs characteristics, BPM, Artists, Genre, Length (and many more) to find that amount of closest neighbors to the new song which would then correctly point whom the most attractive users are. However, this approach could be terribly off if the variables for the songs characteristics do not differentiate the neighboring sets. In this case, Deezer might find it fruitful to invest on capturing more data on the songs and therefore being able to come up with important characteristics that do vary across the different song neighborhoods.

### Scaling

<img src="images/CF_RT.png">

**The previous plot shows how the running time of the model (the size of the green bubble) is affected as the number of users and songs are increased**. If we fix the number of users, say at 1000, the plot shows that increasing the number of songs affects the algorithm in $O(n)$. Going from 100 itmes to a 1000 is a 10x increase and 874/66 $\approx$ 13x. Moreover, the same effect is observed when we fix the number of items and then add more users. Going from 1000 users to 5000 is 5x, while 1805/874 $\approx$ 2x. Although, these are a few examples, in general we observed this linear behavior for different tests we performed.

<img src="images/CF_MSE_RT.png">

As seen from the previous plot, **scaling affects the predictive power of our model marginally**. The almost impreceptible effect of adding more songs to the model is congruent with our finding that only 40 songs are needed to achieve a high predictive power. Therefore, adding more songs to the data should not change the size of neighborhoods. On the other hand, adding more users does marginally worsen the MSE.

**To close the scaling discussion, it is worth mentioning that the implementation of our `ItemBasedCF` class does not waste resources precomputing and placing in memory all the correlation matrix** (where some combinations might never be needed). Rather we compute the correlation for each item that is encountered and store the results in memory. However, if we need to fill in all the matrix, then our algorithm sequentially does it by adding items in each iteration. 

---

## Latent Factor Model with Bias

For the model-based approach we chose a latent factor model with a bias term for missing data. **We assume that the values in the rating matrix are not missing at random and thus introduce a bias term for missing data and weights**.

This section is structured in the following way
* Define the latent factor model
* Hyperparameter Tuning
* Scaling
* Final Test


### Model Definition

For the latent factor model we follow the approach of Steck (2010), where a bias term for missing data is imputed and weights for bias training are used. 

In [ ]:
# SVD with bias and regularization
predictions, U,V = matrix_factorization_model.latent_factors_with_bias(ratings2,
                                                            latent_factors=5, 
                                                            bias=0.2, 
                                                            bias_weights=1, 
                                                            regularization=100,
                                                            learning_rate=0.0001, 
                                                            convergence_rate =0.99999
                                                            )

### Hyperparameter Tuning

**In this section we explore the how different values of the hyperparameters affect the model**, we focus on:
* No of latent factors
* Regularization term
* Bias 
* Bias weights

Tuning will take place in two separate steps. First, we **tuned the number of latent factors together with the regularization parameter**. As these two terms both affect an overfitting of the model, we are tuning them jointly. Second, we **jointly tuned the bias parameters and the bias weights**. For the final test, we will use the parameters that performed best on the validation set. The learning rate and convergence criteria was chosen to be 0.0001 and 0.1 respectively, which preliminary tests have shown to be a good trade-off between runtime and model-improvement. 

#### Tuning of Regularization and Number of Latent Factors

For latent_factors we chose values below 25, as first test have shown that the error is growing drastically for larger latent_factors. For regularization we chose a broad range of values. Furthermore, for initial bias and bias weights, we chose a bias of 0.6, which is close to the mean of the values and bias weights of 0.20, which is close to the fill rate of the matrix. 

In [ ]:
# Parameter Space
latent_factors = [1,2,3,4,5,6,7,8,9,10,12,15,17,20,25]
regularization= [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]

# Lists to store results from each round
ame_errors1 = []
mse_errors1 = []

# Iterate over parameter space
for l in latent_factors: 
    mse, ame = matrix_factorization_model.hyper_parameter_tuning(ratings2,
                                                                DataModel,
                                                                bias= 0.5, 
                                                                bias_weights = 0.2, 
                                                                regularization = regularization,
                                                                latent_factors = l)
    ame_errors1.append(ame)
    mse_errors1.append(mse)

![](regularization-latent-factors.png)

<img src="images/LF_K.png" alt="Drawing" style="width: 450px;"/>

The plot shows, that a parameter of lamba = 50 and number of latent factors = 12 yields the most accurate prediction of the test set. However, we picked a lambda of 5 and number of latent factors of 6 to proceed, as runtime will be faster and the difference in accuracy does not vary significantly.

#### Tuning of Bias and Bias Weights

**For tunning the bias and bias weights we chose any value that each of the weights could take**. While theoretically a negative bias or larger than 1 can be imagined, it would not have an interpretation, as hit rate can not be negative or larger than one. Thus we restricted the search space to parameters between 0 and 1. Bias weights must have to be positive and should be rather small as for high bias weights the model will just learn the bias terms. 

In [ ]:
#Parameter Space 
bias_weights = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.0]
bias = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.0]

#Lists for storing the error per round
ame_errors2 = []
mse_errors2 = []
i = 0

# iterate over bias terms
for b in bias_weights: 
    mse, ame = matrix_factorization_model.hyper_parameter_tuning(ratings2,
                                                                DataModel,
                                                                bias=bias, 
                                                                bias_weights = b, 
                                                                regularization = 5,
                                                                latent_factors = 6)
    ame_errors2.append(ame)
    mse_errors2.append(mse)
    i += 10

<img src="images/bias-bias-weights.png" alt="Drawing" style="width: 450px;"/>

**The plot shows, that low bias parameters in general perform better than high bias terms**. Furthermore, models with lower bias weights perform better than those with higher weights. The following plots shows the lines for bias = 0.6 and bias = 0.7, for more detailed accuracy.

<img src="images/bias-selected.png" alt="Drawing" style="width: 450px;"/>

From this plot it can be derived that a model with bias = 0.6 and bias weight of 0.1 performs best on the validation set. Thus we choose this parameter for our final model test.

#### Scaling users: Accuracy and Run-Time
**To understand how the model is affected by scaling, we calculated the MSE and the runtime depending on the number of users in the rating matrix**. To increase robustnesses of our results, we calculated the mean of 4 different train-validation splits. This is important, as for small number of users the split between test and validation set may largely affect the error.

In [ ]:
# Define scaling steps
scale = range(500,len(ratings2),500)
ratings3 = matrix_creation.hit_rate_matrix_50_50(data)
ratings3.sample(frac=1)
mse_scale = []
runtime_scale = []

# Loop over Training sets with different sizes
for s in scale: 
    ratings_scale = ratings3[:s]
    DataModel = ModelTester.ModelTester()
    DataModel.fit_transform(ratings_scale, verbose = False)
    mse_list = []
    time_list = []
    
    #evaluate metric on data set 4 times to increase robustness of the results
    for i in range(4):
        DataModel.shuffle_cv()
        time_beg = time.time()
        mse, ame = matrix_factorization_model.hyper_parameter_tuning(ratings_scale,
                                                                DataModel,
                                                                bias=0.5, 
                                                                bias_weights = 0.1, 
                                                                regularization = [1],
                                                                latent_factors = 6)
        mse_list.append(mse)
        time_end = time.time()
        time_list.append(time_end - time_beg)
    runtime_scale.append(np.array(time_list).mean())
    mse_scale.append(np.array(mse_list).mean())

At first we analyzed runtime. It is **increasing linearly with adding users**. This makes intuitively sense, as the algorithm should have an complexity of $O(n^{2})$ for the whole matrix. 

<img src="images/scaling-runtime.png" alt="Drawing" style="width: 450px;"/>

That the **MSE increases when more users are added**, was not expected by intuition. However, the increase in MSE might be due to the fact, that interactions between the number of latent factors, regularization and the tuning of bias and bias weights exist. It might be that for more users a more *complex* model is needed, which means more latent factors. 

<img src="images/scaling-mse.png" alt="Drawing" style="width: 450px;"/>

### Final Result: Best Hyperparameter Selection

For the final results we picked the values that yielded the lowest MSE for the validation set. Thus the values are: 
* bias = 0.5
* bias_weights = 0.1
* number of latent_factors = 6
* regularization = 0.6

Our final result for the Latent Factors model yields a **MSE of 0.139.** 

In [ ]:
predictions, U,V = matrix_factorization_model.latent_factors_with_bias(ratings2,
                                                                latent_factors=6, 
                                                                bias=0.5, 
                                                                bias_weights=0.25, 
                                                                regularization=0.6,
                                                                learning_rate=0.0001, 
                                                                convergence_rate =0.99999
                                                                )

---

## Conclusions: Results and Open Questions

**Our analysis set to answer whether Deezer could improve Flow's average hit rate curve** displayed in the first section of the notebook. An improvement on this curve will result in a higher accuracy for the less active users and a general reduction on the variance of this metric. **We found that Deezer could do so by first narrowing its scope** to the most relevant songs in the data **and then allocating its content from an item-based Collaborative Filtering model**. By strategically narrowing down the scope, the recommender systems become less prone to errors since they are not force to make predictions for all the (user, song) combinations but rather for the combinations that are most robust in the data. We found that this worked particularly well but also that novelty and serendipity were potentially preserved. This is the case since the set of most relevant songs was homogeneous throughout the user base. Hence, by focusing on a subset we are not necessarily disregarding potential good recommendations. Moreover, constructing an item-based CF system yielded the best results on the data. **This model obtained a MSE of less than 0.11 whereas the baseline models were around 0.19 - 0.28 and the latent factor model around 0.13**. 

**Following is a set of open questions that we would like to investigate more in depth**:

* What is the potential impact on the user experience for following our narrowing strategy?
* How would our results qualitatively change as more relevant songs are added to the model?
* Would the combination of the our Neighborhood and Latent Factor yield better results?